# Word Embedding Preprocessing

In [1]:
import torch 
import torch.nn as nn
import fasttext as ft
import math
import bcolz
import numpy as np
import pickle

In [2]:
words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(np.zeros(1), rootdir=f'6B.50.dat', mode='w')

In [3]:
with open(f'glove.6B.50d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)

In [4]:
vectors = bcolz.carray(vectors[1:].reshape((-1, 50)), rootdir=f'6B.50.dat', mode='w')
vectors.flush()
pickle.dump(words, open(f'6B.50_words.pkl', 'wb'))
pickle.dump(word2idx, open(f'6B.50_idx.pkl', 'wb'))

In [5]:
vectors = bcolz.open(f'6B.50.dat')[:]
words = pickle.load(open(f'6B.50_words.pkl', 'rb'))
word2idx = pickle.load(open(f'6B.50_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

In [26]:
# To find the index of a word in vocabulary
word2idx['that']

12

In [6]:
with open('vocab.50K.en') as f:
    vece = f.read()
    english_vocab = vece.split('\n')
    
with open('vocab.50K.de') as f:
    vecd = f.read()
    german_vocab = vecd.split('\n')

In [7]:
matrix_len = len(english_vocab)
weights_matrix = np.zeros((matrix_len, 50))
words_found = 0


for i, word in enumerate(english_vocab):
    try: 
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(50, ))  

In [10]:
weights_matrix.shape, words_found

((50001, 50), 28742)

# Sentence Preprocessing

In [18]:
english_sentence_vectors = []
with open(f'train.en', 'rb') as f:
    for l in f:
        line = l.decode()        
        english_sentence_vectors.append(line)

In [20]:
english_sentence_vectors = english_sentence_vectors[:2000]

In [21]:
len(english_sentence_vectors)

2000